In [1]:
import pandas as pd

# Mapping short names to full names (used in the stats file)
short_to_full = {
    "49ers": "San Francisco 49ers",
    "Bears": "Chicago Bears",
    "Bengals": "Cincinnati Bengals",
    "Bills": "Buffalo Bills",
    "Broncos": "Denver Broncos",
    "Browns": "Cleveland Browns",
    "Buccaneers": "Tampa Bay Buccaneers",
    "Cardinals": "Arizona Cardinals",
    "Chargers": "Los Angeles Chargers",
    "Chiefs": "Kansas City Chiefs",
    "Colts": "Indianapolis Colts",
    "Commanders": "Washington Commanders",
    "Cowboys": "Dallas Cowboys",
    "Dolphins": "Miami Dolphins",
    "Eagles": "Philadelphia Eagles",
    "Falcons": "Atlanta Falcons",
    "Giants": "New York Giants",
    "Jaguars": "Jacksonville Jaguars",
    "Jets": "New York Jets",
    "Lions": "Detroit Lions",
    "Packers": "Green Bay Packers",
    "Panthers": "Carolina Panthers",
    "Patriots": "New England Patriots",
    "Raiders": "Las Vegas Raiders",
    "Rams": "Los Angeles Rams",
    "Ravens": "Baltimore Ravens",
    "Saints": "New Orleans Saints",
    "Seahawks": "Seattle Seahawks",
    "Steelers": "Pittsburgh Steelers",
    "Texans": "Houston Texans",
    "Titans": "Tennessee Titans",
    "Vikings": "Minnesota Vikings"
}

# Path to your local data folder
data_path = "/Users/usmanasghar23/CSProjects/College/UIUC/416/D3/Data"

# Load CSV files
playoff_df = pd.read_csv(f"{data_path}/nfl_playoff_appearances_2002_2024.csv")
stats_df = pd.read_csv(f"{data_path}/nfl_team_stats_2002-2024.csv")

# Apply short-to-full mapping
stats_df['home'] = stats_df['home'].map(short_to_full)
stats_df['away'] = stats_df['away'].map(short_to_full)

# Drop rows where mapping failed
stats_df = stats_df.dropna(subset=['home', 'away'])

# Normalize historical names to current names
team_name_fixes = {
    "Oakland Raiders": "Las Vegas Raiders",
    "San Diego Chargers": "Los Angeles Chargers",
    "St. Louis Rams": "Los Angeles Rams",
    "Washington Redskins": "Washington Commanders",
    "Washington Football Team": "Washington Commanders"
}
stats_df['home'] = stats_df['home'].replace(team_name_fixes)
stats_df['away'] = stats_df['away'].replace(team_name_fixes)
playoff_df['Team'] = playoff_df['Team'].replace(team_name_fixes)

# Build season summary (regular season only)
records = []
for season in stats_df['season'].unique():
    season_df = stats_df[
        (stats_df['season'] == season) & 
        (stats_df['week'].astype(str).str.isnumeric())  # Exclude playoffs
    ]
    teams = set(season_df['home']).union(set(season_df['away']))
    for team in teams:
        games_home = season_df[season_df['home'] == team]
        games_away = season_df[season_df['away'] == team]
        wins = ((games_home['score_home'] > games_home['score_away']).sum() +
                (games_away['score_away'] > games_away['score_home']).sum())
        losses = ((games_home['score_home'] < games_home['score_away']).sum() +
                  (games_away['score_away'] < games_away['score_home']).sum())
        pf = games_home['score_home'].sum() + games_away['score_away'].sum()
        pa = games_home['score_away'].sum() + games_away['score_home'].sum()
        records.append({
            'Team': team, 'Season': season,
            'Wins': wins, 'Losses': losses,
            'PointsFor': pf, 'PointsAgainst': pa
        })

team_stats_df = pd.DataFrame(records)

# Melt playoff data for merging
playoff_long_df = playoff_df.melt(id_vars='Team', var_name='Season', value_name='MadePlayoffs')
playoff_long_df['Season'] = playoff_long_df['Season'].astype(int)

# Merge both
merged_df = pd.merge(team_stats_df, playoff_long_df, on=['Team', 'Season'], how='left')
merged_df['MadePlayoffs'] = merged_df['MadePlayoffs'].fillna(0).astype(int)

# Export to CSV
final_path = f"{data_path}/nfl_team_season_summary_2002_2024.csv"
merged_df.to_csv(final_path, index=False)

print(f"✅ File saved to: {final_path}")


FileNotFoundError: [Errno 2] No such file or directory: '/Users/usmanasghar23/CSProjects/College/UIUC/416/D3/Data/nfl_playoff_appearances_2002_2024.csv'